In [ ]:
# 한국인 강의음성 전처리 및 csv 파일 생성
# https://aihub.or.kr/aidata/30708/download

# 압축을 풀고 "한국어 강의 음성" 폴더의 이름을 KlecSpeech로 변경한다.
# 추가 히스토리
# 2022-06-15 array 추가

In [ ]:
import re

def preprocessing(text):
    # 괄호(문자)/(발음) 일 때, 발음만 살리는 코드
    """
    괄호 및 괄호 안에 있는 문장을 모두 벗겨내기
    \([^\)]*\)\/|\(|\)

    조건 1: (문장)/ 을 삭제한다. 
    \([^\)]*\)    (사과)/(배) (1000원)/(천원)
    \/            뒤에 /를 추가하면 뒤에 /가 없는 (배) 와 (천원)은 조건 탈락

    |             조건 1과 조건 2를 OR로 검출

    조건 2 :  ( 과 )를 검출
    \(|\)  
    """  
    re_text = re.sub(r'\([^\)]*\)\/|\(|\)', '', text)
    # re_text = re.sub(r'[^\w\s]', '', re_text) # 괄호 삭제
    # hangul = re.compile('[^ ㄱ-ㅣ가-힣+]') # 한글과 띄어쓰기를 제외한 모든 글자
    # re_text = hangul.sub('', re_text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    # re_text = re_text.strip()
#     print(re_text)
    return re_text

test_text = "안녕하세요. (사과)/(배)는 (1000원)/(천원) 입니다."
test_ret = preprocessing(test_text)
print(test_ret)

In [ ]:
def preprocessings(text):
    # 괄호(문자)/(발음) 일 때, 발음만 살리는 코드
    re_text = re.sub(r'\([^\)]*\)\/|\(|\)', '', text)
    return re_text

test_text = "안녕하세요. (사과)/(배)는 (1000원)/(천원) 입니다."
test_ret = preprocessings(test_text)
print(test_ret)

안녕하세요. 배는 천원 입니다.


## 오디오 데이터 추가

In [ ]:
import json
import os
import glob
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import librosa

In [ ]:
text_dict = {'filename':[],'text':[],'array':[]}

path='./dataset/KlecSpeech/Validation/'

# path = Path('./한국어 강의 음성/Validation/')

# wav 파일 기준으로 text 파일이름이 동일, 확장자는 wav, txt로 각각 다름
files = [f for f in glob.glob(path + "**/*.wav", recursive=True)]

# 테스트 코드
# files = files[0:3]
# print(files)

for i in tqdm(range(len(files))):
    txt_file = files[i].replace(".wav", ".txt")
    files[i] = files[i].replace("\\", "/")

#     print(txt_file)
    if not os.path.exists(txt_file):
        print("The file does not exist", txt_file)
    
    with open(txt_file ,'r',encoding='utf8') as f:
        text = f.read()
        
        # 텍스트 전처리
        text = preprocessing(text)

        # 항목들 값을 저장 \
        text_dict['filename'].append(files[i]) # filename
        text_dict['text'].append(text)
        
        # 각 wav파일을 16000으로 나눠서 저장
        array,_ = librosa.load(files[i],sr=16000)
        text_dict['array'].append(list(array))

    if i%500==499:
        n=i//500
        df = pd.DataFrame(text_dict)
        df.to_csv(f'klecspeech_ko_500_{n:03d}.csv',index=False,sep='\t')
        print(f'{n:03d} complete')

        # text_dict을 초기화
        text_dict = {'filename':[],'text':[],'array':[]}
        
# df = pd.DataFrame(text_dict)
# df.head()

  2%|█▏                                                                          | 508/32971 [00:33<6:26:36,  1.40it/s]

000 complete


  3%|██▎                                                                       | 1006/32971 [01:47<18:49:18,  2.12s/it]

001 complete


  5%|███▎                                                                      | 1502/32971 [03:47<54:37:50,  6.25s/it]

...

003 complete


  8%|█████▌                                                                   | 2500/32971 [10:32<118:48:28, 14.04s/it]

004 complete


  8%|██████▎                                                                    | 2802/32971 [12:50<2:18:18,  3.64it/s]


KeyboardInterrupt: 

In [ ]:
csv_df = pd.read_csv('klecspeech_ko_000.csv',sep='\t')
csv_df.head()

In [ ]:
for i in range(100):
    print(text_dict['src'][i], text_dict['text'][i])

In [ ]:
df.to_csv('order_speech_ko.csv')

In [ ]:
df['text'].unique()
# df.read_csv('order_speech_ko.csv')
# result = [i for i in my_list if i]

In [ ]:
df.info()

In [ ]:
df.describe()

## 텍스트 누락 확인

In [ ]:
df['text'].min()

In [ ]:
df['text_len'] = df['text'].map(len)

df[['text_len']].idxmin()

In [ ]:
df[['text_len']].idxmin()

In [ ]:
df['text'][22410]

In [ ]:
print(df['src'][22410])

In [ ]:
# 찾아보면 #NAME이라고 되어 있음.